# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Dataset
from azureml.train.automl.utilities import get_primary_metrics
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
import os
import joblib

ModuleNotFoundError: ignored

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl_hr_analytics'

experiment=Experiment(ws, experiment_name)

In [ ]:
ws

In [ ]:
cpu_cluster_name = "compute-cluster"
vm_size = "Standard_D3_V2"
try:
    compute_target = ComputeTarget(workspace = ws, name= cpu_cluster_name)
    print("Found already existing")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

In [ ]:
dataset = Dataset.get_by_name(ws, name='data')
dataset.to_pandas_dataframe()

In [ ]:
dataset =dataset.drop_columns(["enrollee_id","city"])

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "task": "classification", 
    "primary_metric": "accuracy",
    "training_data": dataset,
    "label_column_name": "target"    
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    n_cross_validations= 6,
    enable_early_stopping= True,
    compute_target= compute_target,
    max_cores_per_iteration= -1,
    max_concurrent_iterations= 4,
    featurization= "auto",
    
    **automl_settings
    )

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(automl_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
automl_best_run, model= automl_run.get_output()
automl_best_run

In [ ]:
model._final_estimator

In [ ]:
.makedirs('./outputs', exist_ok=True)

In [ ]:
#TODO: Save the best model
joblib.dump(model, filename="outputs/automl_model.joblib")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

In [ ]:
service.delete()